## Residual Network 구현 및 학습

In [ ]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [ ]:
EPOCHS = 10

## Residual Unit 구현

In [ ]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        if filter_in == filter_out : 
            self.identity = lambda x : x
        else : 
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding = 'same')

    def call(self, x, training = False, mask=None):
        h = self.bn1(x, training = training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(x, training = training)
        h = tf.nn.relu(h)
        h = self.conv2(h)

        return self.identity(x) + h

## Residual Layer 구현

In [ ]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = list()

        for f_in, f_out in zip([filter_in]+list(filters), filters):
            # zip([16,32,32,32], [32,32,32]) 이면, 앞의 것의 마지막은 탈락하게 되어 (16,32), (32,32), (32,32)만 나오게 됨
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))

    def call(self, x, training = False, mask=None):
        for unit in self.sequence:
            x = unit(x, training = training)
            return x 

## 모델 정의

In [ ]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3,3), padding= 'same', activation='relu') # 28x28x8
        self.res1 = ResnetLayer(8, (16,16), (3,3)) # 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2,2)) # 14x14x16

        self.res2 = ResnetLayer(16, (32,32), (3,3)) # 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2,2)) # 7x7x32

        self.res3 = ResnetLayer(32, (64,64), (3,3)) # 7x7x64
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')


    def call(self, x, training=False, mask=None):
        x = self. conv1(x)

        x = self.res1(x, training = training)
        x = self.pool1(x)

        x = self.res2(x, training = training)
        x = self.pool2(x)

        x = self.res3(x, training = training)

        x = self.flatten(x)
        x = self.dense1(x)

        return self.dense2(x)

## 학습, 테스트 루프 정의

In [ ]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training = True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training = False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [ ]:
# Create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [ ]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.10623688995838165, Accuracy: 96.86833953857422, Test Loss: 0.07261963188648224, Test Accuracy: 97.79000091552734
Epoch 2, Loss: 0.05403891205787659, Accuracy: 98.44833374023438, Test Loss: 0.06780624389648438, Test Accuracy: 98.18000030517578
Epoch 3, Loss: 0.04158129543066025, Accuracy: 98.7699966430664, Test Loss: 0.07971996814012527, Test Accuracy: 97.93000030517578
Epoch 4, Loss: 0.03557207062840462, Accuracy: 99.0, Test Loss: 0.03659195825457573, Test Accuracy: 98.94000244140625
Epoch 5, Loss: 0.030882811173796654, Accuracy: 99.14666748046875, Test Loss: 0.03359392285346985, Test Accuracy: 99.04000091552734
Epoch 6, Loss: 0.027939407154917717, Accuracy: 99.19000244140625, Test Loss: 0.05585072189569473, Test Accuracy: 98.83000183105469
Epoch 7, Loss: 0.022480780258774757, Accuracy: 99.37999725341797, Test Loss: 0.05737195163965225, Test Accuracy: 98.66999816894531
Epoch 8, Loss: 0.02275233343243599, Accuracy: 99.35832977294922, Test Loss: 0.05702725052833557, Test